Training Notebook for the G2Net competition. This implements a Bi-directional GRU using Keras, using preprocessed spectrogram.

This uses Yasufumi Nakama's spectrogram preprocessing notebooks and datasets:
* Train: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-train), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-train-images)
* Test: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-test), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-test-images)

In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
# !pip install -q nnAudio -qq
# import torch
# from nnAudio.Spectrogram import CQT1992v2
# Q_TRANSFORM = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=32)

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, df, directory, batch_size=32, random_state=42, shuffle=True, target=True, ext='.npy'):
        np.random.seed(random_state)
        
        self.directory = directory
        self.df = df
        self.shuffle = shuffle
        self.target = target
        self.batch_size = batch_size
        self.ext = ext
#         self.q_transform = CQT1992v2(
#             sr=2048, fmin=20, fmax=1024, hop_length=32
#         )
        
        self.on_epoch_end()
    
    def __len__(self):
        return np.ceil(self.df.shape[0] / self.batch_size).astype(int)
    
    
    
    def __get_qtransform(self, x):
#         image = []
#         for i in range(3):
        waves = x / np.max(x)
        waves = torch.from_numpy(waves).float()
        image = self.q_transform(waves).squeeze().numpy()
#         image.append(channel)
#         image = np.stack(image)
        
        return image
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        batch = self.df[start_idx: start_idx + self.batch_size]
        
        signals = []

        for fname in batch.id:
            path = os.path.join(self.directory, fname + self.ext)
            data = np.load(path)
#             data = self.__get_qtransform(data)
            signals.append(data)
        
        signals = np.stack(signals).astype('float32')
        signals = np.expand_dims(signals, axis=3)
        
        if self.target:
            return signals, batch.target.values
        else:
            return signals
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
def build_model():
#     inputs = layers.Input(shape=(27, 128, 1))
    inputs = layers.Input(shape=(27, 128, 1))

    conv0 = layers.Conv2D(filters = 32, kernel_size = (3,5), strides = (1,1), activation = 'relu')
    norm0 = layers.BatchNormalization()
    conv1 = layers.Conv2D(filters = 32, kernel_size = (3,5), strides = (1,1), activation = 'relu')
    norm1 = layers.BatchNormalization()
    pool0 = layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 3), padding='valid')
    
    conv2 = layers.Conv2D(filters = 64, kernel_size = (3,1), strides = (1,1), activation = 'relu')
    norm2 = layers.BatchNormalization()
    conv3 = layers.Conv2D(filters = 64, kernel_size = (3,1), strides = (1,1), activation = 'relu')
    norm3 = layers.BatchNormalization()
    pool1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')
    
    conv4 = layers.Conv2D(filters = 128, kernel_size = (1,5), strides = (1,1), activation = 'relu')
    norm4 = layers.BatchNormalization()
    conv5 = layers.Conv2D(filters = 128, kernel_size = (1,5), strides = (1,1), activation = 'relu')
    norm5= layers.BatchNormalization()
    pool2 = layers.MaxPooling2D(pool_size=(2, 2), strides=(3, 3), padding='valid')
    
    conv6 = layers.Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = 'relu')
    norm6 = layers.BatchNormalization()
    conv7 = layers.Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = 'relu')
    norm7 = layers.BatchNormalization()
    pool3 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
    
    flat = layers.Flatten()
    
    gru0 = layers.Bidirectional(layers.GRU(256, return_sequences=True))
    drop0 = layers.SpatialDropout1D(0.25)
    gru1 = layers.Bidirectional(layers.GRU(256, return_sequences=True))
    drop1 = layers.SpatialDropout1D(0.25)
    
    avgpool = layers.GlobalAveragePooling1D(name='avg_pool')
    maxpool = layers.GlobalMaxPooling1D(name='max_pool')
    
        
    x = conv0(inputs)
    x = norm0(x)
    x = conv1(x)
    x = norm1(x)
    x = pool0(x)
    
    x = conv2(x)
    x = norm2(x)
    x = conv3(x)
    x = norm3(x)
    x = pool1(x)
    
    x = conv4(x)
    x = norm4(x)
    x = conv5(x)
    x = norm5(x)
    x = pool2(x)
    
    x = conv6(x)
    x = norm6(x)
    x = conv7(x)
    x = norm7(x)
    x = pool3(x)
    
#     x = tf.squeeze(x, [1])
    
    x = flat(x)
    x = tf.expand_dims(x, axis=1)
    
    x = gru0(x)
    x = drop0(x)
    x = gru1(x)
    x = drop1(x)

    x = tf.keras.layers.Concatenate()([avgpool(x), maxpool(x)])
    
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(1, activation="sigmoid", name="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    return model

model = build_model()
model.compile("adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
model.summary()

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
# df_split = np.array_split(train, 7)
# train = df_split[0]
sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
train.head()

In [ ]:
sample_df = train.sample(frac=1).reset_index(drop=True)

split = int(sample_df.shape[0] * 0.8)

train_df = sample_df[:split]
valid_df = sample_df[split:]

# train_df = train_df[:10000]
# valid_df = valid_df[:2000]

In [ ]:
train_dset = CustomDataset(
    train_df, '../input/g2net-n-mels-128-train-images', batch_size=64)

valid_dset = CustomDataset(
    valid_df, '../input/g2net-n-mels-128-train-images', batch_size=64, shuffle=False)

test_dset = CustomDataset(
    sub, "../input/g2net-n-mels-128-test-images", batch_size=64, target=False, shuffle=False)

In [ ]:
# import matplotlib.pyplot as plt

# for elem in train_dset:
#     plt.imshow(elem[0][0])
#     plt.show()
# #   print(elem[0].shape)

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    "model_weights.h5", save_best_only=True, save_weights_only=True,
)

train_history = model.fit(
    train_dset, 
    use_multiprocessing=True, 
    workers=4, 
    epochs=20,
    validation_data=valid_dset,
    callbacks=[ckpt],
)

In [ ]:
model.load_weights('model_weights.h5')

In [ ]:
y_pred = model.predict(
    test_dset, use_multiprocessing=True, workers=4, verbose=1
)

In [ ]:
sub['target'] = y_pred
sub.to_csv('submission.csv', index=False)
sub.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
# !pip install kaggle --upgrade
# !kaggle competitions submit -c g2net-gravitational-wave-detection -f /kaggle/working/submission.csv -m "Message"